In [1]:
from chart_creator import ChartCreator as cc
from exchange.board import *

In [2]:
history = load_file('../DATA/MERGE.log.gz')
history

In [3]:
tick_vol = 50

df = history._filter_execute(history.log_data).copy()
df

,action,time,sequence,price,volume,checksum
601,6,2020-05-05 00:00:02.514339,1719606531,9641490,0.12,NaN
602,6,2020-05-05 00:00:02.536735,1719606532,9641490,0.01,NaN
709,4,2020-05-05 00:00:02.836836,1719606533,9641860,0.02,NaN
710,4,2020-05-05 00:00:02.836836,1719606534,9641860,0.03,NaN
711,6,2020-05-05 00:00:02.961833,1719606535,9641490,0.01,NaN
...,...,...,...,...,...,...
6950506,6,2020-05-05 05:25:20.376481,1720144492,9857690,0.02,NaN
6950544,6,2020-05-05 05:25:20.485851,1720144493,9857690,0.03,NaN
6950545,6,2020-05-05 05:25:20.485851,1720144494,9857470,0.03,NaN
6950546,6,2020-05-05 05:25:20.485851,1720144495,9857190,0.01,NaN


In [4]:
max_vol = df[VOLUME].sum()
start = int(max_vol + tick_vol) - max_vol

ticks = np.arange(start, max_vol, tick_vol)
ticks

array([   49.47546861,    99.47546861,   149.47546861,   199.47546861,
         249.47546861,   299.47546861,   349.47546861,   399.47546861,
         449.47546861,   499.47546861,   549.47546861,   599.47546861,
         649.47546861,   699.47546861,   749.47546861,   799.47546861,
         849.47546861,   899.47546861,   949.47546861,   999.47546861,
        1049.47546861,  1099.47546861,  1149.47546861,  1199.47546861,
        1249.47546861,  1299.47546861,  1349.47546861,  1399.47546861,
        1449.47546861,  1499.47546861,  1549.47546861,  1599.47546861,
        1649.47546861,  1699.47546861,  1749.47546861,  1799.47546861,
        1849.47546861,  1899.47546861,  1949.47546861,  1999.47546861,
        2049.47546861,  2099.47546861,  2149.47546861,  2199.47546861,
        2249.47546861,  2299.47546861,  2349.47546861,  2399.47546861,
        2449.47546861,  2499.47546861,  2549.47546861,  2599.47546861,
        2649.47546861,  2699.47546861,  2749.47546861,  2799.47546861,
      

In [5]:
df['sum'] = df[VOLUME].cumsum()
bins = pd.cut(df['sum'], ticks)

bins

601        NaN
602        NaN
709        NaN
710        NaN
711        NaN
          ... 
6950506    NaN
6950544    NaN
6950545    NaN
6950546    NaN
6950614    NaN
Name: sum, Length: 525058, dtype: category
Categories (743, interval[float64]): [(49.475, 99.475] < (99.475, 149.475] < (149.475, 199.475] < (199.475, 249.475] ... (36999.475, 37049.475] < (37049.475, 37099.475] < (37099.475, 37149.475] < (37149.475, 37199.475]]

In [6]:
df = df.groupby(bins).agg({'time': 'last', 'price': ['first', 'last', 'max', 'min'], 'volume': 'sum'}, axis=1)
df

time    price                    \
                                             last    first     last      max   
sum                                                                            
(49.475, 99.475]       2020-05-05 00:00:26.071374  9652580  9650970  9654850   
(99.475, 149.475]      2020-05-05 00:00:37.824813  9651320  9655320  9657200   
(149.475, 199.475]     2020-05-05 00:00:59.800413  9655320  9663820  9663820   
(199.475, 249.475]     2020-05-05 00:01:17.244252  9663830  9661130  9663940   
(249.475, 299.475]     2020-05-05 00:01:33.239526  9659560  9669500  9672000   
...                                           ...      ...      ...      ...   
(36949.475, 36999.475] 2020-05-05 05:22:56.190107  9846500  9845850  9848250   
(36999.475, 37049.475] 2020-05-05 05:23:37.651684  9845780  9850500  9851200   
(37049.475, 37099.475] 2020-05-05 05:24:06.870029  9850500  9850270  9851840   
(37099.475, 37149.475] 2020-05-05 05:24:36.628011  9850270  9852980  9854560   
(37149.475, 37199.475] 2020-05-05 05:25:10.268643  9852980  9855330  9858240   

                                    volume  
                            min        sum  
sum                                         
(49.475, 99.475]        9648610  49.984566  
(99.475, 149.475]       9648600  49.743541  
(149.475, 199.475]      9655260  50.201884  
(199.475, 249.475]      9655660  50.098946  
(249.475, 299.475]      9659560  49.890856  
...                         ...        ...  
(36949.475, 36999.475]  9842350  50.440551  
(36999.475, 37049.475]  9844260  49.455626  
(37049.475, 37099.475]  9846920  50.406312  
(37099.475, 37149.475]  9848790  50.066894  
(37149.475, 37199.475]  9851380  49.979546  

[743 rows x 6 columns]

In [7]:
df.columns = ['time_stamp', 'open', 'close', 'high', 'low', 'volume']
df

,time_stamp,open,close,high,low,volume
sum,,,,,,
"(49.475, 99.475]",2020-05-05 00:00:26.071374,9652580,9650970,9654850,9648610,49.984566
"(99.475, 149.475]",2020-05-05 00:00:37.824813,9651320,9655320,9657200,9648600,49.743541
"(149.475, 199.475]",2020-05-05 00:00:59.800413,9655320,9663820,9663820,9655260,50.201884
"(199.475, 249.475]",2020-05-05 00:01:17.244252,9663830,9661130,9663940,9655660,50.098946
"(249.475, 299.475]",2020-05-05 00:01:33.239526,9659560,9669500,9672000,9659560,49.890856
...,...,...,...,...,...,...
"(36949.475, 36999.475]",2020-05-05 05:22:56.190107,9846500,9845850,9848250,9842350,50.440551
"(36999.475, 37049.475]",2020-05-05 05:23:37.651684,9845780,9850500,9851200,9844260,49.455626
"(37049.475, 37099.475]",2020-05-05 05:24:06.870029,9850500,9850270,9851840,9846920,50.406312


In [8]:
df.reset_index(drop=True, inplace=True)
df

,time_stamp,open,close,high,low,volume
0,2020-05-05 00:00:26.071374,9652580,9650970,9654850,9648610,49.984566
1,2020-05-05 00:00:37.824813,9651320,9655320,9657200,9648600,49.743541
2,2020-05-05 00:00:59.800413,9655320,9663820,9663820,9655260,50.201884
3,2020-05-05 00:01:17.244252,9663830,9661130,9663940,9655660,50.098946
4,2020-05-05 00:01:33.239526,9659560,9669500,9672000,9659560,49.890856
...,...,...,...,...,...,...
738,2020-05-05 05:22:56.190107,9846500,9845850,9848250,9842350,50.440551
739,2020-05-05 05:23:37.651684,9845780,9850500,9851200,9844260,49.455626
740,2020-05-05 05:24:06.870029,9850500,9850270,9851840,9846920,50.406312
741,2020-05-05 05:24:36.628011,9850270,9852980,9854560,9848790,50.066894


In [9]:
df.index.name = 'time'
df = df[['time_stamp', 'open', 'close', 'high', 'low', 'volume']]
df

,time_stamp,open,close,high,low,volume
time,,,,,,
0,2020-05-05 00:00:26.071374,9652580,9650970,9654850,9648610,49.984566
1,2020-05-05 00:00:37.824813,9651320,9655320,9657200,9648600,49.743541
2,2020-05-05 00:00:59.800413,9655320,9663820,9663820,9655260,50.201884
3,2020-05-05 00:01:17.244252,9663830,9661130,9663940,9655660,50.098946
4,2020-05-05 00:01:33.239526,9659560,9669500,9672000,9659560,49.890856
...,...,...,...,...,...,...
738,2020-05-05 05:22:56.190107,9846500,9845850,9848250,9842350,50.440551
739,2020-05-05 05:23:37.651684,9845780,9850500,9851200,9844260,49.455626
740,2020-05-05 05:24:06.870029,9850500,9850270,9851840,9846920,50.406312


In [10]:
cc.initialize()
cc.set_xaxis(start=0, end=0, is_unixtime=True)
cc.add_subchart(ax=0, label="USD", grid=True)

In [11]:
df

,time_stamp,open,close,high,low,volume
time,,,,,,
0,2020-05-05 00:00:26.071374,9652580,9650970,9654850,9648610,49.984566
1,2020-05-05 00:00:37.824813,9651320,9655320,9657200,9648600,49.743541
2,2020-05-05 00:00:59.800413,9655320,9663820,9663820,9655260,50.201884
3,2020-05-05 00:01:17.244252,9663830,9661130,9663940,9655660,50.098946
4,2020-05-05 00:01:33.239526,9659560,9669500,9672000,9659560,49.890856
...,...,...,...,...,...,...
738,2020-05-05 05:22:56.190107,9846500,9845850,9848250,9842350,50.440551
739,2020-05-05 05:23:37.651684,9845780,9850500,9851200,9844260,49.455626
740,2020-05-05 05:24:06.870029,9850500,9850270,9851840,9846920,50.406312


In [17]:
#df.index.name='unixtime'
df.columns = ['unixtime', 'open', 'close', 'high', 'low', 'volume']

df

,unixtime,open,close,high,low,volume
unixtime,,,,,,
0,2020-05-05 00:00:26.071374,9652580,9650970,9654850,9648610,49.984566
1,2020-05-05 00:00:37.824813,9651320,9655320,9657200,9648600,49.743541
2,2020-05-05 00:00:59.800413,9655320,9663820,9663820,9655260,50.201884
3,2020-05-05 00:01:17.244252,9663830,9661130,9663940,9655660,50.098946
4,2020-05-05 00:01:33.239526,9659560,9669500,9672000,9659560,49.890856
...,...,...,...,...,...,...
738,2020-05-05 05:22:56.190107,9846500,9845850,9848250,9842350,50.440551
739,2020-05-05 05:23:37.651684,9845780,9850500,9851200,9844260,49.455626
740,2020-05-05 05:24:06.870029,9850500,9850270,9851840,9846920,50.406312


In [20]:
import mplfinance as mpf


In [21]:
mpf.plot(df)


TypeError: Expect data.index as DatetimeIndex

In [ ]:
cc.set_ohlcv_df(df)


In [16]:
cc.create_chart('./test.png', 'png')



[create_chart] plot data does not exist.
